В качестве несложного прогноза будем делать прогноз закрытия цены на 5 дней вперед при помощи линейной регрессии.

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import datetime as dt
import warnings
import pandas_datareader.data as web
from plotly.subplots import make_subplots
warnings.filterwarnings('ignore')

In [2]:
Bitcoin = pd.read_csv("data/BTC.csv", index_col=0, parse_dates=['timeOpen', 'timestamp'], dayfirst=True)
Bitcoin

,timeOpen,open,close,volume,marketCap,timestamp,high,low
0,2016-12-19 00:00:00+00:00,790.692017,792.713989,7.488640e+07,1.272509e+10,2016-12-19 23:59:59.999000+00:00,793.611023,790.320007
1,2016-12-20 00:00:00+00:00,792.247009,800.875977,9.962930e+07,1.285747e+10,2016-12-20 23:59:59.999000+00:00,801.336975,791.497009
2,2016-12-21 00:00:00+00:00,800.643982,834.281006,1.555760e+08,1.339545e+10,2016-12-21 23:59:59.999000+00:00,834.281006,799.405029
3,2016-12-22 00:00:00+00:00,834.179993,864.539978,2.000270e+08,1.388299e+10,2016-12-22 23:59:59.999000+00:00,875.781982,834.148987
4,2016-12-23 00:00:00+00:00,864.888000,921.984009,2.755640e+08,1.480719e+10,2016-12-23 23:59:59.999000+00:00,925.117004,864.677002
...,...,...,...,...,...,...,...,...
2195,2022-12-23 00:00:00+00:00,16829.643586,16796.952171,1.532927e+10,3.232031e+11,2022-12-23 23:59:59.999000+00:00,16905.219360,16794.459352
2196,2022-12-24 00:00:00+00:00,16796.976620,16847.755265,9.744636e+09,3.241902e+11,2022-12-24 23:59:59.999000+00:00,16864.702879,16793.528185
2197,2022-12-25 00:00:00+00:00,16847.505364,16841.987052,1.165638e+10,3.240932e+11,2022-12-25 23:59:59.999000+00:00,16860.553971,16755.253234
2198,2022-12-26 00:00:00+00:00,16842.249311,16919.805175,1.188696e+10,3.256043e+11,2022-12-26 23:59:59.999000+00:00,16920.123640,16812.370076


In [3]:
projection_Bitcoin = 5
#creation of a new column with a name prediction
Bitcoin['prediction'] = Bitcoin[['close']].shift(-projection_Bitcoin)

In [4]:
visualize = cycle(['Open','Close','High','Low','Prediction'])

fig = px.line(Bitcoin, x=Bitcoin['timeOpen'], y=[Bitcoin['open'], Bitcoin['close'], Bitcoin['high'], Bitcoin['low'], Bitcoin['prediction']],
             labels={'Date': 'Date','value':'Price'})
fig.update_layout(title_text='Bitcoin', font_size=15, font_color='black',legend_title_text='Parameters')
fig.for_each_trace(lambda t: t.update(name = next(visualize)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

In [5]:
X = np.array(Bitcoin[['close']])
X = X[:-projection_Bitcoin]
y_ = Bitcoin['prediction'].values
y_ = y_[:-projection_Bitcoin]
x_train, x_test, y_train, y_test = train_test_split(X,y_,test_size=0.15)
print(y_)

[  898.82202148   896.1829834    907.60998535 ... 16841.98705228
 16919.80517493 16717.17398013]


In [6]:
linReg_Bitcoin = LinearRegression()
linReg_Bitcoin.fit(x_train,y_train)
linReg_confidence_Bitcoin = linReg_Bitcoin.score(x_test, y_test)

print(linReg_confidence_Bitcoin)

0.9852674573778469


In [7]:
from sklearn.linear_model import Lasso
Ls=Lasso()
Ls.fit(x_train,y_train)
ypred=Ls.predict(x_test)
print(Ls.score(x_test,y_test))

0.9852674573336706


In [8]:
from sklearn.tree import DecisionTreeRegressor
mytre=DecisionTreeRegressor(max_depth=1)
mytre.fit(x_train,y_train)
ypredd=mytre.predict(x_test)
mytre.score(x_test,y_test)

0.8176298329231008

In [9]:
x_projection_Bitcoin = np.array(Bitcoin[['close']])[-projection_Bitcoin:]
print(x_projection_Bitcoin)

[[16796.95217122]
 [16847.75526515]
 [16841.98705228]
 [16919.80517493]
 [16717.17398013]]


In [10]:
from sklearn import svm
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import VotingRegressor

In [11]:
model2=KNeighborsRegressor(n_neighbors=5)
model3=DecisionTreeRegressor()
model4= VotingRegressor([
                         ('KNN',model2),('tree',DecisionTreeRegressor())
                         ])
for model in (model2,model3, model4):
    model.fit(x_train,y_train)
    print(model.__class__.__name__, model.score(x_test,y_test))

KNeighborsRegressor 0.9826457905194339
DecisionTreeRegressor 0.9723742916379847
VotingRegressor 0.9802803323163348


В целом здесь можно заюзать любые регрессионные модели

### Визуализация

In [12]:
START_DATE = "2022-01-01" # Начальная дата для визуализации
RSI_PERIODS = 7 # Период для расчет RSI

crypto_list = ['BTC', 'ETH', 'BNB', 'XRP', 'DOGE', 'ADA', 'MATIC', 'DAI', 'DOT', 'LTC']

# Считываем данные
def df_loader(lst , start_date = "2021-01-01"):
    filenames = []
    all_df = pd.DataFrame()
    for name in lst:
        df = pd.read_csv("data/"+ name + ".csv", parse_dates=["timeOpen", "timestamp"])
        df = df [df["timeOpen"] >= start_date]
        df.index = df["timeOpen"]
        df.drop(labels = [df.columns[0],df.columns[6],df.columns[1]] , axis = 1 , inplace = True)
        df['name']=name
        all_df = pd.concat([all_df,df], ignore_index=False)
        filenames.append("data/"+ name + ".csv")
    return all_df , filenames

In [13]:
def RSI(df, periods = 14, ema = True):
    close_delta = df['close'].diff().dropna()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
	    # экспоненциальная скользящая средняя
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Скользящее среднее
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [14]:
all_df , filenames = df_loader(crypto_list , start_date=START_DATE)

df_vizualize = []
for file in crypto_list:
    symbol = file
    temp_df = pd.DataFrame(all_df[all_df["name"] == symbol])
    temp_df.drop(columns= ["name"] ,inplace = True)
    temp_df["close_rsi"] = RSI(temp_df, periods=RSI_PERIODS)
    temp_df["high_rsi"] = 20
    temp_df["low_rsi"] = 70
    exec('%s = temp_df.copy()' % file.lower())
    df_vizualize.append(temp_df)


In [33]:
fig = make_subplots(rows=3, 
                    cols=2,
                    shared_xaxes=True,
                    specs=[[{"rowspan": 2}, {"rowspan": 2}], [{"rowspan": 1}, {"rowspan": 1}] , [{},{}]]
                    
                   )
date_buttons = [
{'step': "all", 'label': "All time"},
{'count':  1, 'step': "year", 'stepmode': "backward", 'label': "Last Year"},
{'count':  1, 'step': "year", 'stepmode': "todate", 'label': "Current Year"},
{'count':  1, 'step': "month", 'stepmode': "backward", 'label': "Last 2 Months"},
{'count':  1, 'step': "month", 'stepmode': "todate", 'label': "Current Month"},
{'count':  7, 'step': "day", 'stepmode': "todate", 'label': "Current Week"},
{'count':  4, 'step': "day", 'stepmode': "backward", 'label': "Last 4 days"},
{'count':  1, 'step': "day", 'stepmode': "backward", 'label': "Today"},
               ]
buttons = []
i = 0
j=0
COUNT = 8
vis = [False] * len(crypto_list) * COUNT
for df in df_vizualize:
    for k in range(COUNT):
        vis[j+k] = True
    buttons.append({ 'label' : crypto_list[i],
                     'method' : 'update',
                     'args'   : [{'visible' : vis},
                                {'title'  : crypto_list[i] + ' Charts and Indicators'}
                                ] }
                  )
    i+=1
    j+=COUNT
    vis = [False] * len(crypto_list)* COUNT
for df in df_vizualize:
    fig.add_trace(
        go.Candlestick(x= df.index,
                       open  = df['open'],
                       high  = df['high'],
                       low   = df['low'],
                       close = df['close'],
                       showlegend =False
                                ),
                  row=1, 
                  col=1)
    fig.add_trace(
        go.Bar(x =df.index ,
               y=df["volume"],
               showlegend =False, 
               marker_color='#1f77b4'),
        row=3, 
        col=1)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['close'],
                   mode='lines',
                  showlegend =False,
                   line=dict(color="#d62728", width=4)),
        row=1, 
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index,
                   y=df['low'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False,                   
                   line=dict(width=2, color='pink', dash='dash')),
        row=1, 
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index, 
                   y=df['high'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False, 
                   line=dict(width=2, color='pink', dash='dash')),
        row=1, 
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['close_rsi'],
                   mode='lines',
                  showlegend =False,
                   line=dict(color="#1f77b4", width=1)),
        row=3, 
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index,
                   y=df['low_rsi'],
                   fill='tonexty', 
                   mode='lines',
                   showlegend =False,                   
                   line=dict(width=2, color='aquamarine', dash='dash')),
        row=3, 
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index, 
                   y=df['high_rsi'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False, 
                   line=dict(width=2, color='aquamarine', dash='dash')),
        row=3, 
        col=2)
    
    
fig.update_xaxes(
        tickfont = dict(size=15, family = 'monospace', color='#B8B8B8'),
        tickmode = 'array',
        ticklen = 6,
        showline = False,
        showgrid = True,
        gridcolor = '#595959',
        ticks = 'outside')
fig.update_layout(
    spikedistance=100, 
    xaxis_rangeslider_visible=False,
    hoverdistance=1000)
fig.update_xaxes(
    showspikes=True,  
    spikesnap="cursor",
    spikemode="across"
                )
fig.update_yaxes(
    showspikes=True, 
    spikesnap='cursor',
    spikemode="across"                 
                )
fig.update_yaxes(
        tickfont = dict(size=15, family = 'monospace', color='#B8B8B8'),
        tickmode = 'array',
        showline = False,
        ticksuffix = '$',
        showgrid = True,
        gridcolor = '#595959',
        ticks = 'outside')
fig.update_layout(
                  width=1120,
                  height=650,
                  font_family   = 'monospace',
                  xaxis         = dict(rangeselector = dict(buttons = date_buttons)),
                  updatemenus   = [dict(type = 'dropdown',
                                        x = 1.0,
                                        y = 1.108,
                                        showactive = True,
                                        active = 2,
                                        buttons = buttons)],
                  title         = dict(text = '<b>Крипто Дашборд<b>',
                                       font = dict(color = '#000000' ,size = 22), 
                                       x = 0.50),
                  font          = dict(color="blue"), 
                  annotations   = [
                                  dict( text = "<b>Выбрать криптовалюту<b>",
                                        font = dict(size = 20 , color = "#000000"),
                                        showarrow=False,
                                        x = 1.02, 
                                        y = 1.20,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Свечи <b>",
                                        font = dict(size = 20,  color = "#000000"),
                                        showarrow=False,
                                        x = 0.18, 
                                        y = 0.285,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Цена<b>",
                                        font = dict(size = 20,  color = "#000000"),
                                        showarrow=False,
                                        x = 0.82, 
                                        y = 0.285,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Объем<b>",
                                        font = dict(size = 20,  color = "#000000"),
                                        showarrow=False,
                                        x = 0.18, 
                                        y = -0.58,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>RSI<b>",
                                        font = dict(size = 20,  color = "#000000"),
                                        showarrow=False,
                                        x = 0.8, 
                                        y = -0.58,
                                        xref = 'paper', yref = "paper",
                                        align = "left")
                  ],
                template= "seaborn"
                )    
for i in range(0, 10*COUNT):
    fig.data[i].visible = False
for i in range(COUNT):
    fig.data[i].visible = True
fig.layout["xaxis"]["rangeslider"]["visible"] = False
fig.layout["xaxis2"]["rangeslider"]["visible"] = False
fig.layout["xaxis5"]["rangeslider"]["visible"] = True
fig.layout["xaxis6"]["rangeslider"]["visible"] = True
fig.layout["xaxis5"]["rangeslider"]["borderwidth"] = 4
fig.layout["xaxis6"]["rangeslider"]["borderwidth"] = 4
fig.layout["xaxis5"]["rangeslider"]["bordercolor"] = "#1f77b4"
fig.layout["xaxis6"]["rangeslider"]["bordercolor"] = "#1f77b4"
fig.layout["yaxis6"]["ticksuffix"] = ""
fig.layout["yaxis6"]["range"] = [10,100]
fig.show()